In [60]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import pymysql
from sqlalchemy import create_engine, text

In [69]:
def combine_columns(df):
    df["String Combination"] = ('In round ' + df['round'].astype(str) + ', with a value of ' + df['clue_value'].astype(str) +
    ', a daily double value of ' + df['daily_double_value'].astype(str) + ', in the category ' + df['category'].astype(str) +
    ', the question was ' + df['answer'].astype(str) + ' and the answer was ' + df['question'].astype(str) + '. This was on ' +
    df['air_date'].astype(str) + '.')
    df.loc[df['comments'].str.contains(r"[a-zA-Z]"), 'String Combination'] += ' Comments: ' + df['comments'].astype(str) + '.'
    df.loc[df['notes'].str.contains(r"[a-zA-Z]"), 'String Combination'] += ' Notes: ' + df['notes'].astype(str) + '.'
    return df

In [70]:
load_dotenv()  # Load from .env file

def create_connection():
    try:
        connection = pymysql.connect(
            host=os.getenv("DB_HOST"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            database = os.getenv("DB_NAME")
        )
        if connection:
            print("Connection successful!")
        return connection
    except pymysql.Error as e:
        print(f"Connection error: {e}")
        return None

def get_db_creds():
    host = os.getenv("DB_HOST")
    user = os.getenv("DB_USER")
    password = os.getenv("DB_PASSWORD")
    name = os.getenv("DB_NAME")
    table = os.getenv("TABLE_NAME")
    return host, user, password, name, table

In [71]:
query = "SELECT * FROM main_data LIMIT 100"

conn = create_connection()

df = pd.read_sql_query(query, conn)

Connection successful!


/var/folders/zj/r53y5rb179z438lmdfg069y80000gn/T/ipykernel_97587/2568179190.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [72]:
df.head()

,id,round,clue_value,daily_double_value,category,comments,answer,question,air_date,notes,created_at,string_combination
0,1,1,100,0,LAKES & RIVERS,,River mentioned most often in the Bible,the Jordan,1984-09-10,,2025-04-25 19:55:39,None
1,2,1,200,0,LAKES & RIVERS,,Scottish word for lake,loch,1984-09-10,,2025-04-25 19:55:39,None
2,3,1,400,0,LAKES & RIVERS,,American river only 33 miles shorter than the ...,the Missouri,1984-09-10,,2025-04-25 19:55:39,None
3,4,1,500,0,LAKES & RIVERS,,"Worlds largest lake, nearly 5 times as big as ...",the Caspian Sea,1984-09-10,,2025-04-25 19:55:39,None
4,5,1,100,0,INVENTIONS,,Marconis wonderful wireless,a radio,1984-09-10,,2025-04-25 19:55:39,None


In [74]:
df = combine_columns(df)
print(df['String Combination'][0])

In round 1, with a value of 100, a daily double value of 0, in the category LAKES & RIVERS, the question was River mentioned most often in the Bible and the answer was the Jordan. This was on 1984-09-10.


In [55]:
host, user, password, name, table = get_db_creds()
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{name}")
with engine.connect() as connection:
    df = pd.read_sql(query, connection)

In [ ]:
df["String Combination"] = 'In round ' + df['round'].astype(str) + ', with a value of ' + df['clue_value'].astype(str) //
    ', a daily double value of ' + df['daily_double_value'].astype(str) + ', in the category ' + df['category'].astype(str) + //
    ', the question was ' + df['answer'].astype(str) + ' and the answer was ' + df['question'].astype(str) + '. This was on ' // +
    df['air_date'].astype(str) + '.'

In [ ]:
concat_query = "UPDATE main_data SET combination_string = CONCAT('In round ', round, ', with a value of ', clue_value, ', a daily double value of ', daily_double_value, ', in the category ', category, ', the question was ', answer, ' and the answer was ', question, '. This was on ', air_date, '. Comments: ', comments, '. Notes: ', notes, '.');"
add_column_query = "ALTER TABLE main_data ADD COLUMN combination_string TEXT;"
read_query = "SELECT * FROM main_data LIMIT 100;"

with engine.connect() as connection:
    connection.execute(text(add_column_query))
    connection.execute(text(concat_query))
    df = pd.read_sql(query, connection)


KeyboardInterrupt: 

In [57]:
connection.close()

In [43]:
df[df['comments'].str.contains(r"[a-zA-Z]")]

,id,round,clue_value,daily_double_value,category,comments,answer,question,air_date,notes,created_at,String Combination
494,495,2,200,0,1789,(Alex: ...a good year.),"Washington proclaimed Nov. 26, 1789 this first...",Thanksgiving,1984-09-21,,2025-04-25 19:55:39,"In round 2, with a value of 200"
495,496,2,400,0,1789,(Alex: ...a good year.),Why April 28th was a bad day for Capt. Bligh,the day of the mutiny on the Bounty,1984-09-21,,2025-04-25 19:55:39,"In round 2, with a value of 400"
496,497,2,1000,0,1789,(Alex: ...a good year.),"Now totaling over $1 trillion, it began 11 day...",the national debt,1984-09-21,,2025-04-25 19:55:39,"In round 2, with a value of 1000"
507,508,2,200,0,HOMONYMS,(Alex: And for the benefit of our folks at hom...,Hindu hierarchy or a plays actors,a caste (cast),1984-09-21,,2025-04-25 19:55:39,"In round 2, with a value of 200"
508,509,2,400,0,HOMONYMS,(Alex: And for the benefit of our folks at hom...,Near or a purchase,by (buy),1984-09-21,,2025-04-25 19:55:39,"In round 2, with a value of 400"
...,...,...,...,...,...,...,...,...,...,...,...,...
515868,515869,2,400,0,I + 4,(Ken: I will be the first letter of these 5-le...,"Typically found at the end of a book, its an a...",index,2024-07-26,,2025-04-25 20:00:23,"In round 2, with a value of 400"
515869,515870,2,800,0,I + 4,(Ken: I will be the first letter of these 5-le...,This Texas-based company has been making its i...,Igloo,2024-07-26,,2025-04-25 20:00:23,"In round 2, with a value of 800"
515870,515871,2,1200,0,I + 4,(Ken: I will be the first letter of these 5-le...,"Last name of Max, son of a famous actor father...",Irons,2024-07-26,,2025-04-25 20:00:23,"In round 2, with a value of 1200"
515871,515872,2,1600,0,I + 4,(Ken: I will be the first letter of these 5-le...,William Butler Yeats got nationalistic as he u...,Irish,2024-07-26,,2025-04-25 20:00:23,"In round 2, with a value of 1600"
